In [2]:
#get data
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from spotipy import util

#analytics
import pandas as pd

import sys

In [3]:
client_id = '759349ca4662426990ede422e2a1efdb' #Need to create developer profile
client_secret = 'b59771d6de47460babce1ce563df31ed'
username = 'rldilley-us' #Store username
redirect_uri='http://localhost:8080/'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)#Create manager for ease
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Get Data

In [10]:
def get_all_tracks_and_features(username, sp):
    '''
    return df of all track info and features of all user liked songs and playlist songs
    '''
    
    playlist_tracks_df = get_all_tracks_from_all_user_playlists(username, sp) #get all playlist tracks info df
    
    track_features_df = get_all_tracks_features(playlist_tracks_df)
    
    df_all_tracks = pd.merge(playlist_tracks_df, track_features_df,  how='left', left_on=['name','track_id'], right_on = ['name','id'])
    
    return df_all_tracks

def get_all_tracks_from_all_user_playlists(username, sp):
    '''
    return a df of all track info from all user playlists
    '''
    
    playlist_dict = get_all_user_playlists(username,sp) #get a dict of all playlist names and playlist ids
    
    #create dataframe for all playlist track info
    cols = ['name', 'track_id', 'artists', 'album_name', 'date_added', 'images']
    all_tracks_df = pd.DataFrame( columns = cols)
    for name,playlist_id in playlist_dict.items(): #loop through all playlists
        
        #get df of all tracks in a playlist and add to playlist track info df
        songs = get_all_tracks_from_playlist(playlist_id,username, sp)
        playlist_tracks_df = get_tracks_info_df(songs, username, sp)
        all_tracks_df = pd.concat([all_tracks_df, playlist_tracks_df])
    
    return all_tracks_df
        
def get_all_user_playlists(username, sp):
    '''
    return a playlist dictionary with key, playlist name and value, playlist id
    '''
    
    playlists = sp.user_playlists(username) #call api to get all user playlists
    playlist_dict = {}
    for playlist in playlists['items']:
        playlist_dict[playlist['name']] = playlist['id']
    
    return playlist_dict

def get_all_tracks_from_playlist(playlist_id, username, sp):
    '''
    return list of tracks from a playlist
    '''

    playlist = sp.user_playlist_tracks(username,playlist_id) #call api to get all tracks in a playlist

    #Extract list of songs/tracks
    tracks = playlist["items"]
    while playlist['next']:
        playlist = sp.next(playlist)
        tracks.extend(playlist['items'])
    
    return tracks

def get_tracks_info_df(tracks_lst, username, sp):
    '''
    input list of tracks and return df of all tracks info
    '''
    
    #create tracks info df and loop through all tracks and append to track info df
    cols = ['name', 'track_id', 'artists', 'album_name', 'date_added', 'images']
    tracks_info_df = pd.DataFrame( columns = cols)
    for i in range(0, len(tracks_lst)):
        
        name =  tracks_lst[i]['track']['name']
        trackid = tracks_lst[i]['track']['id']

        artists = []
        for x in tracks_lst[i]['track']['artists']:
            artists.append(x['name'])

        imgs = []
        for y in tracks_lst[i]['track']['album']['images']:
            imgs.append(y['url'])

        tracks_info_df = tracks_info_df.append({'name': name, 'track_id': trackid, 'artists': artists, 'album_name': tracks_lst[i]['track']['album']['name'], 
                                      'date_added': tracks_lst[i]['added_at'], 'images':imgs}
                                       , ignore_index = True)
    return tracks_info_df

def get_all_user_liked_tracks(username, sp):
    songs = sp.current_user_saved_tracks()
    liked_songs = songs["items"]

    while liked_songs['next']:
        songs = sp.next(songs)
        liked_songs.extend(songs['items'])
    
    return liked_songs

def get_all_tracks_features(df):
    
    #create feature df
    cols = ['name', 'id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
    features_df = pd.DataFrame(columns = cols)
    
    for index, row in df.iterrows():
        track_id = row['track_id']
        features = sp.audio_features(track_id) #call api
        audio_features = features[0]
        features_df = features_df.append({'name': row['name'], 'id': track_id, 'danceability': audio_features['danceability'], 'energy': 
                                         audio_features['energy'], 'key': audio_features['key'], 'loudness': audio_features['loudness']
                                         , 'mode': audio_features['mode'], 'speechiness': audio_features['speechiness'], 'acousticness':
                                         audio_features['acousticness'], 'instrumentalness':audio_features['instrumentalness'] 
                                         ,'liveness': audio_features['liveness'], 'valence': audio_features['valence'], 'tempo':
                                         audio_features['tempo'], 'duration_ms': audio_features['duration_ms'], 'time_signature': 
                                         audio_features['time_signature']}
                                        , ignore_index = True)
    
    return features_df


In [11]:
df = get_all_tracks_from_all_user_playlists(username, sp)

In [12]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3355 entries, 0 to 206
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        3355 non-null   object
 1   track_id    3355 non-null   object
 2   artists     3355 non-null   object
 3   album_name  3355 non-null   object
 4   date_added  3355 non-null   object
 5   images      3355 non-null   object
dtypes: object(6)
memory usage: 183.5+ KB


,name,track_id,artists,album_name,date_added,images
0,Beach Baby,7oRNU7Tmek0nm2lqyDojoG,[Bon Iver],Blood Bank,2021-10-13T15:19:25Z,[https://i.scdn.co/image/ab67616d0000b273a069f...
1,Free,4b57xU7rviFrRgilxMjDJb,[Sylvan Esso],Free Love,2021-10-13T15:20:15Z,[https://i.scdn.co/image/ab67616d0000b273bffb4...
2,Us and Them,1TKTiKp3zbNgrBH2IwSwIx,[Pink Floyd],The Dark Side of the Moon,2021-10-13T15:20:56Z,[https://i.scdn.co/image/ab67616d0000b273ea7ca...
3,Lotus Blossom,33p7jMh4ThQdZ3fe6scpfI,[Michael Franks],One Bad Habit,2021-10-13T15:21:56Z,[https://i.scdn.co/image/ab67616d0000b273f4c0c...
4,Wait,3rTnGUeDrnZV22DvRuUuXr,[M83],"Hurry Up, We're Dreaming",2021-10-13T15:23:07Z,[https://i.scdn.co/image/ab67616d0000b273fff2c...


In [13]:
df.to_csv("user_tracks.csv")

In [ ]:
track_id = '7oRNU7Tmek0nm2lqyDojoG'

#start = time.time()
#analysis = sp.audio_analysis(track_id)
#delta = time.time() - start
#print(json.dumps(analysis, indent=4))


track = sp.track(track_id)
print(track)

In [1]:
scope = 'user-library-read'
#sp = spotipy.Spotipy(auth_manager=SpotifyOAuth(scope=scope))
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)#Create manager for ease
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

songs = sp.current_user_saved_tracks()
liked_songs = songs["items"]

while liked_songs['next']:
    songs = sp.next(songs)
    liked_songs.extend(songs['items'])
    
#return liked_songs


NameError: name 'SpotifyClientCredentials' is not defined

NameError: name 'do_something' is not defined